In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

False 2.2.0+cpu


In [2]:
from src import API
API.Trainer.main(stage = 0 , resume = 0 , checkname= 1)

24-06-03 20:56:51|MOD:display     |: Model Specifics:
24-06-03 20:56:51|MOD:display     |: Start Process [Data] at Mon Jun  3 20:56:51 2024!


--Process Queue : Data + Fit + Test
--Model_name is set to tra_day.1!
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=2) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=10,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=2) , display batch progress bar
Callback : StatusDisplay(verbosity=2) , display epoch and event information
Callback : SpecCB_TRA() , in TRA fill [y] [hist_loss] in batch_data.kwargs , update hist_loss in data.buffer
{'random_seed': None,
 'model_name': 'tra_day.1',
 

24-06-03 20:56:54|MOD:display     |: Finish Process [Data], Cost 2.5 Secs
24-06-03 20:56:54|MOD:display     |: Start Process [Fit] at Mon Jun  3 20:56:54 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


FirstBite Ep#  0 : loss  1.02127, train 0.00336, valid 0.00583, best 0.0058, lr1.3e-03
FirstBite Ep#  5 : loss  0.92681, train 0.09793, valid 0.10683, best 0.1068, lr2.5e-03
FirstBite Ep# 10 : loss  0.89070, train 0.12985, valid 0.12485, best 0.1268, lr1.9e-03
FirstBite Ep# 15 : loss  0.86994, train 0.14678, valid 0.13158, best 0.1357, lr1.0e-07
FirstBite Ep# 20 : loss  0.86932, train 0.14639, valid 0.13518, best 0.1428, lr9.4e-04
FirstBite Ep# 25 : loss  0.85684, train 0.15872, valid 0.14360, best 0.1448, lr3.1e-04
FirstBite Ep# 30 : loss  0.85480, train 0.16009, valid 0.14112, best 0.1448, lr1.6e-04
FirstBite Ep# 35 : loss  0.85277, train 0.16190, valid 0.14551, best 0.1455, lr3.1e-04
Reset learn rate and scheduler at the end of epoch 39 , effective at epoch 40
FirstBite Ep# 40 : loss  0.84942, train 0.16507, valid 0.14449, best 0.1461, lr1.3e-03
FirstBite Ep# 45 : loss  0.86654, train 0.14755, valid 0.13605, best 0.1472, lr2.5e-03
FirstBite Ep# 50 : loss  0.85167, train 0.16227, val

In [1]:
from src import API
API.DataAPI.reconstruct_train_data()
API.Trainer.update_models()

Use device name: NVIDIA GeForce RTX 4090
predict is False , Data Processing start!
4 datas :['y', 'day', '30m', 'risk']
y blocks loading start!
 --> labels blocks reading [ret10_lag] DataBase...... finished! Cost 20.05 secs
 --> labels blocks reading [ret20_lag] DataBase...... finished! Cost 18.36 secs
 --> labels blocks merging (2)...... finished! Cost 2.97 secs
 --> models blocks reading [risk_exp] DataBase...... finished! Cost 66.12 secs
y blocks loading finished! Cost 118.05 secs
y blocks process...... finished! Cost 45.72 secs
y blocks masking...... finished! Cost 0.81 secs
y blocks saving ...... finished! Cost 4.98 secs
y blocks norming...... finished! Cost 0.00 secs
y finished! Cost 169.72 Seconds
--------------------------------------------------------------------------------
day blocks loading start!
 --> trade blocks reading [day] DataBase...... finished! Cost 30.05 secs
day blocks loading finished! Cost 30.08 secs
day blocks process...... finished! Cost 3.38 secs
day blocks 